In [1]:
neg_dir = "/home/vedic/ISI/review_polarity/txt_sentoken/neg/"
pos_dir = "/home/vedic/ISI/review_polarity/txt_sentoken/pos/"

In [2]:
import os

In [3]:
neg_revs = []
pos_revs = []

for file in os.listdir(neg_dir):
    text = open(neg_dir + file, 'r' )
    text = text.read()
    neg_revs.append(text)
    
for file in os.listdir(pos_dir):
    text = open(pos_dir + file, 'r' )
    text = text.read()
    pos_revs.append(text)

In [4]:
import string
import nltk
from nltk.corpus import stopwords

In [5]:
def clean_text(text):
    tokens = text.split()
    tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word not in string.punctuation]
    tokens = [word for word in tokens if word.isalpha()]
    stop = set(nltk.corpus.stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop]
    tokens = [word for word in tokens if len(word) > 1]
    text = " ".join(tokens)
    return text

In [6]:
neg_rev_clean = []
pos_rev_clean = []

for rev in neg_revs:
    neg_rev_clean.append(clean_text(rev))
    
for rev in pos_revs:
    pos_rev_clean.append(clean_text(rev))

In [7]:
labels = ([0 for _ in range(len(neg_rev_clean))] + [1 for _ in range(len(pos_rev_clean))])

In [8]:
reviews = neg_rev_clean + pos_rev_clean

In [9]:
from sklearn.model_selection import train_test_split

x_test, x_train, y_test, y_train = train_test_split(reviews, labels, test_size = 0.9)

In [11]:
from keras.preprocessing.text import Tokenizer

Using TensorFlow backend.


In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x_train)

In [13]:
vocab_size = len(tokenizer.word_index) + 1

In [14]:
max_length = max([len(s.split()) for s in x_train])

In [15]:
encoded = tokenizer.texts_to_sequences(x_train)

In [16]:
from keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(encoded, maxlen=max_length, padding = 'post' )

In [17]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(Conv1D(filters=32, kernel_size=8, activation= 'relu' ))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation= 'relu' ))
model.add(Dense(1, activation= 'sigmoid' ))

In [19]:
model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1299, 100)         3583500   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 1292, 32)          25632     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 646, 32)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 20672)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                206730    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 3,815,873
Trainable params: 3,815,873
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.fit(x_train, y_train, epochs=10, verbose=1)

Epoch 1/10
1800/1800 [==============================] - 12s 7ms/step - loss: 0.6918 - acc: 0.5056
Epoch 2/10
1800/1800 [==============================] - 11s 6ms/step - loss: 0.6280 - acc: 0.6100
Epoch 3/10
1800/1800 [==============================] - 12s 6ms/step - loss: 0.4202 - acc: 0.8967
Epoch 4/10
1800/1800 [==============================] - 12s 7ms/step - loss: 0.3443 - acc: 0.9578
Epoch 5/10
1800/1800 [==============================] - 11s 6ms/step - loss: 0.3163 - acc: 0.9678: 1s - loss: 0.3209 - ac
Epoch 6/10
1800/1800 [==============================] - 11s 6ms/step - loss: 0.2987 - acc: 0.9761: 2s - loss: 0.2991 -
Epoch 7/10
1800/1800 [==============================] - 12s 6ms/step - loss: 0.2841 - acc: 0.9778
Epoch 8/10
1800/1800 [==============================] - 12s 6ms/step - loss: 0.2731 - acc: 0.9772
Epoch 9/10
1800/1800 [==============================] - 12s 6ms/step - loss: 0.2628 - acc: 0.9778
Epoch 10/10
1800/1800 [==============================] - 12s 6ms/step - l

In [22]:
encoded = tokenizer.texts_to_sequences(x_test)
x_test  = pad_sequences(encoded, maxlen=max_length, padding = 'post' )

In [24]:
model.evaluate(x_test, y_test, verbose=2)

[0.5109314346313476, 0.83]